# Radio Access TagNet Software Image Directory Listing

In [ ]:
from __future__ import print_function
from builtins import *                  # python3 types
from time import sleep
from datetime import datetime
import struct as pystruct
from binascii import hexlify, unhexlify
import os
import sys

In [ ]:
!pwd
%autosave 0

In [ ]:
sys.path.insert(0,'../tagfuse/tagfuse')
print(os.getcwd())
from radioutils import radio_start, path2list, radio_send_msg, radio_receive_msg
from radiofile import file_get_bytes, file_put_bytes, file_update_attrs, dblk_put_note
from radioimage import im_get_dir, im_put_file, im_delete_file, im_get_file

In [ ]:
sys.path.insert(0, '../tagnet')
from tagnet import TagMessage, TagPoll, TagGet, TagPut, TagDelete, TagHead
from tagnet import TagName
from tagnet import TagTlv, TagTlvList, tlv_types

In [ ]:
import datetime
print('Test Start Time: {}'.format(datetime.datetime.now()))

##  Start up Radio

In [ ]:
radio = radio_start()

## Check for Command Error

In [ ]:
status = radio.get_chip_status()
if (status.chip_pend.CMD_ERROR):
    print(status)

## Image Info description
Image Description Information stored in the Image File

In [ ]:
#  IMAGE_INFO provides information about a Tag software image. This data is
#  embedded in the image itself. The IMAGE_META_OFFSET is the offset into
#  the image where image_info lives in the image.  It directly follows the
#  exception vectors which are 0x140 bytes long.
# 
#  This struct will have to change, If MSP432 vector table length changes.
# 
IMAGE_INFO_SIG = 0x33275401
IMAGE_META_OFFSET = 0x140
IMAGE_INFO_DEFAULT = [IMAGE_INFO_SIG, 0x20000, (0x140*2)+0x1c, 0, 0, 257, 2, 3, ' '  * 40, ' '  * 40, ' '  * 30, 0xde, 2]
#
# Struct created for accessing image info (little indian)
# sig, image_start, imagelength, vector_chk, image_chk, im_build, im_minor, im_major, main_tree, aux_tree, build_time, im_rev, im_model = image_info
#
IM_FIELDS = '<LLLLLHBB44s44s30sBB'
image_info_struct = pystruct.Struct(IM_FIELDS)
IMAGE_MIN_SIZE  =  (IMAGE_META_OFFSET + image_info_struct.size)

In [ ]:
# write out simple default binary input file for testing purposes
#
def default_image(filename):
    if not os.path.isfile(filename):
        with open(filename,'wb') as outfile:
            buf = bytearray(IMAGE_META_OFFSET)
            for x in range(1,IMAGE_META_OFFSET): buf[x] = x & 0x7f
            outfile.write(buf)
            outfile.write(bytearray(image_info_struct.pack(*IMAGE_INFO_DEFAULT)))
            for i in range(64):
                for x in range(256): buf[x] = x & 0x7f
                outfile.write(buf)

In [ ]:
def image_file_info(filename):
    infile = open(filename, 'rb')
    infile.seek(0, 2) # seek to the end
    file_size = infile.tell()
    if file_size < IMAGE_MIN_SIZE: raise RadioLoadException("input file too short")
    infile.seek(0, 0)    # seek to the beginnnig
    # get image info from input file and sanity check
    infile.seek(IMAGE_META_OFFSET) # seek to location of image info
    image_info = image_info_struct.unpack(infile.read(image_info_struct.size))
    print("file information")
    sig, image_start, imagelength, vector_chk, image_chk, im_build, im_minor, im_major,\
            main_tree, aux_tree, build_time, im_rev, im_model = image_info
    pstr = "  signature: 0x{:x}, start: 0x{:x}, length: 0x{:x}, vect_chk: 0x{:x}, image_chk: 0x{:x}"
    print(pstr.format(sig, image_start, imagelength, vector_chk, image_chk))
    pstr = "  version: ({}.{}.{}(0x{:x})), rev: {}, model: {}"
    print(pstr.format(im_major, im_minor, im_build, im_build, im_rev, im_model))
    if sig != IMAGE_INFO_SIG: raise RadioLoadException("image metadata is invalid")
    infile.seek(0)    # seek to the beginning
    infile.close()

In [ ]:
image_main    = 'main.bin'
image_test    = '/tmp/test.bin'

In [ ]:
image_file_info(image_main)

In [ ]:
default_image(image_test)
image_file_info(image_test)

## Image Directory and File Information

In [ ]:
# default paramters
MAX_WAIT            = 10
MAX_RECV            = 255
MAX_PAYLOAD         = 254
MAX_RETRIES         = 10
RADIO_POWER         = 100
SHORT_DELAY         = 0

In [ ]:
#"tag"  "sd"    <node_id>   "img"
def get_directory():
    image_manager_name = TagName ([
                TagTlv(tlv_types.NODE_ID, -1),
                TagTlv('tag'),
                TagTlv('sd'),
                TagTlv(0),
                TagTlv('img')])

    dir_info = TagGet(image_manager_name)
    #print(dir_info.name)
    dir_msg = dir_info.build()
    #
    print(len(dir_msg),hexlify(dir_msg))
    radio_send_msg(radio, dir_msg, RADIO_POWER);
    rsp_buf, rssi, status = radio_receive_msg(radio, MAX_RECV, MAX_WAIT)
    if (rsp_buf):
        print(len(rsp_buf),hexlify(rsp_buf))
        rsp_obj = TagMessage(rsp_buf)
        for x in range(0, 8, 2):
            print("state: {}, {}".format(rsp_obj.payload[x+1].value(), rsp_obj.payload[x]))
    else:
        print('timeout')

In [ ]:
#"tag"	"sys"	<node_id>	"which"
def get_version(which):
    sys_name = TagName ([
                TagTlv(tlv_types.NODE_ID, -1),
                TagTlv('tag'),
                TagTlv('sys'),
                TagTlv(which)])
            
    sys_obj = TagGet(sys_name)
#    print(sys_obj.name)
    get_msg = sys_obj.build()
#
    print(len(get_msg),hexlify(get_msg))
    radio_send_msg(radio, get_msg, RADIO_POWER);
    rsp_buf, rssi, status = radio_receive_msg(radio, MAX_RECV, MAX_WAIT)
    if(rsp_buf):
#
        print(len(rsp_buf),hexlify(rsp_buf))
        rsp_obj = TagMessage(rsp_buf)
        print("{} {:^10} state: {}, {}".format(rsp_obj.header.options.param.error_code, \
                                                         which, rsp_obj.payload[1].value(),       \
                                                         rsp_obj.payload[0]))
    sleep(.1)

In [ ]:
STOP

## Get Directory

In [ ]:
get_directory()

## Get Versions

In [ ]:
get_version('active')

In [ ]:
get_version('backup')

In [ ]:
get_version('golden')

In [ ]:
get_version('nib')

In [ ]:
get_version('running')

In [ ]:
rsp_buf=bytearray(unhexlify("2691801a0506ffffffffffff0103746167010373797301066163746976650a0480010200010161"))
rsp_buf=bytearray(unhexlify("1c9088190506ffffffffffff0103746167010273640201000103696d67"))
print(hexlify(rsp_buf))
tn = TagMessage(rsp_buf)
print(tn.name)
print(tn.payload)

## Def Set Active/Backup Version

In [ ]:
#"tag"	"sys"	<node_id>	"which"
def set_version(which, version):
    set_name = TagName ([
                TagTlv(tlv_types.NODE_ID, -1),
                TagTlv('tag'),
                TagTlv('sys'),
                TagTlv(which),
                TagTlv(tlv_types.VERSION, version)])
    set_obj = TagPut(set_name)
#    print(set_obj.name)
    set_msg = set_obj.build()
    radio_send_msg(radio, set_msg, RADIO_POWER);
    rsp_buf, rssi, status = si446x_device_receive_msg(radio, MAX_RECV, 5)
    if(rsp_buf):
#        print(hexlify(rsp_buf))
        rsp_obj = TagMessage(rsp_buf)
#        print(rsp_obj.header)
        if (rsp_obj.payload):
            print("{}: state: {}, {}".format(rsp_obj.header.options.param.error_code, rsp_obj.payload[1].value(), rsp_obj.payload[0]))
        else:
            print("{}".format(rsp_obj.header.options.param.error_code))

## Set Active Version

In [ ]:
set_version('active', (118, 1, 0))

## Set Backup Version

In [ ]:
set_version('backup', (112, 1, 0))

## Reboot Running Version

In [ ]:
set_version('running', (116, 1, 0))

## Reboot NIB

In [ ]:
set_version('nib', (115, 1, 0))

## Reboot Golden

In [ ]:
set_version('golden', (117, 1, 0))

## Delete Version

In [ ]:
#"tag"  "sd"    <node_id>   "img"   <version>
def delete_image(version):
    delete_image_name = TagName ([
                TagTlv(tlv_types.NODE_ID, -1),
                TagTlv('tag'),
                TagTlv('sd'),
                TagTlv(0),
                TagTlv('img'),
                TagTlv(tlv_types.VERSION, version)])
    delete_info = TagDelete(delete_image_name)
    #print(delete_info.header)
    print(delete_info.name)
    delete_msg = delete_info.build()
    radio_send_msg(radio, delete_msg, RADIO_POWER);
    rsp_buf, rssi, status = si446x_device_receive_msg(radio, MAX_RECV, 5)
    if(rsp_buf):
#    print(hexlify(rsp_buf))
        rsp_obj = TagMessage(rsp_buf)
#    print(rsp_obj.header.options.param.error_code)
#    print(rsp_obj.payload)
        if (rsp_obj.payload):
            print("{}: state: {}, {}".format(rsp_obj.header.options.param.error_code, rsp_obj.payload[1].value(), rsp_obj.payload[0]))
        else:
            print("{}".format(rsp_obj.header.options.param.error_code))

In [ ]:
delete_image((383, 2, 0))

## Get Chip Status

In [ ]:
print(radio.get_chip_status())

## Interactive Group Properties

In [ ]:
from ipywidgets import interact
interact(si446x_device_group_fetch_and_decode, group=radio_config_group_ids.encoding)

## Interactive  Command Status Responses

In [ ]:
interact(si446x_device_command_fetch_and_decode, cmd=radio_status_cmd_ids.encoding)

In [ ]:
from datetime import datetime
datetime.now()

## Examine Radio Trace Buffer

In [ ]:
radio.trace.display(radio.trace.filter())